In [1]:
# pip install tensorflow_datasets

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

2023-12-18 12:43:55.231674: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-18 12:43:55.399300: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-18 12:43:55.585346: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 12:43:55.585434: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 12:43:55.603451: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised= True)

In [4]:
train_data, test_data = imdb['train'], imdb['test']

In [5]:
training_sentences= []
training_labels = []
testing_sentences= []
testing_labels = []

for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(str(l.numpy()))

for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(str(l.numpy()))

In [6]:
training_sentences[2345]

'b"This Alec Guinness starrer is a very good fun political satire of corporate industry, and a light eccentric character study as well.<br /><br />The pacing is a bit slow for a comedy, and none of it is really rolling-on-the-floor type funny, except perhaps the sound effects for the experiments. But it does have its amusing moments, and it is very deft in its execution. The big explosions segment is probably the most farcical element.<br /><br />The union procedures are quite droll, very reminiscent of I\'M ALL RIGHT JACK; especially the feminine socialist with a light romantic crush on Guinness\' character. The political machinations actually carry the story. Ernest Thesigner is very notable as a heavy.<br /><br />I don\'t think this one works quite as well as THE LADYKILLERS, or KIND HEARTS AND CORONETS; but even light Ealing comedy is better than nothing."'

In [7]:
training_labels[2345]

'1'

In [8]:
len(training_sentences), len(testing_sentences)

(25000, 25000)

In [9]:
import pandas as pd

In [10]:
df = pd.Series(training_labels)
df.value_counts()

0    12500
1    12500
Name: count, dtype: int64

In [11]:
from collections import Counter
Counter(training_labels)

Counter({'0': 12500, '1': 12500})

In [12]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [13]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'

In [14]:
# pip install keras_preprocessing

In [15]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
word_index;
len( word_index)

86538

In [16]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)
testing_sentences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)

In [17]:
padded.shape

(25000, 120)

In [18]:
testing_padded.shape

(25000, 120)

In [19]:
padded

array([[   0,    0,    0, ...,  873,  144,    9],
       [   0,    0,    0, ...,   31,   30,   46],
       [6174,    1, 4915, ...,    8, 6175,   46],
       ...,
       [7628,   36,   10, ...,  167,    5,   28],
       [2676,   10,  215, ...,    1,   88,   10],
       [3874,    4,   30, ...,    5,  994, 5125]], dtype=int32)

In [20]:
word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 "'s": 20,
 'on': 21,
 'you': 22,
 'not': 23,
 'are': 24,
 'his': 25,
 'he': 26,
 'have': 27,
 'be': 28,
 'one': 29,
 'all': 30,
 'at': 31,
 'by': 32,
 'they': 33,
 'an': 34,
 'who': 35,
 'so': 36,
 'from': 37,
 'like': 38,
 'her': 39,
 "'t": 40,
 'or': 41,
 'just': 42,
 'there': 43,
 'about': 44,
 'out': 45,
 "'": 46,
 'has': 47,
 'if': 48,
 'some': 49,
 'what': 50,
 'good': 51,
 'more': 52,
 'very': 53,
 'when': 54,
 'she': 55,
 'up': 56,
 'can': 57,
 'b': 58,
 'time': 59,
 'no': 60,
 'even': 61,
 'my': 62,
 'would': 63,
 'which': 64,
 'story': 65,
 'only': 66,
 'really': 67,
 'see': 68,
 'their': 69,
 'had': 70,
 'were': 71,
 'me': 72,
 'well': 73,
 'we': 74,
 'than': 75,
 'much': 76,
 'been': 77,
 'get': 78,
 'bad': 79,
 'will': 80,
 'people': 81,
 'do': 82,
 'also': 83,


#### Build the model

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding

In [23]:
model_rnn = Sequential([
Embedding(vocab_size, embedding_dim, input_length=max_length),
SimpleRNN(32),
Dense(10,activation='relu'),
Dense(1, activation = 'sigmoid')
])

model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1568      
                                                                 
 dense (Dense)               (None, 10)                330       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 161909 (632.46 KB)
Trainable params: 161909 (632.46 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
history = model_rnn.fit(padded, training_labels_final, epochs=3, validation_data = (testing_padded, testing_labels_final) )

Epoch 1/3


2023-12-18 12:53:49.656580: W tensorflow/core/framework/op_kernel.cc:1816] OP_REQUIRES failed at cast_op.cc:122 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node binary_crossentropy/Cast defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/dai/anaconda3/envs/NLP/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/dai/anaconda3/envs/NLP/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/home/dai/anaconda3/envs/NLP/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/home/dai/anaconda3/envs/NLP/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/dai/anaconda3/envs/NLP/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/home/dai/anaconda3/envs/NLP/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/home/dai/anaconda3/envs/NLP/lib/python3.11/asyncio/events.py", line 80, in _run

  File 

In [24]:
model.evaluate(x, y_new, batch_size=1)

25/25 [==============================] - 1s 5ms/step - loss: 3.2574 - accuracy: 0.0400


[3.257401704788208, 0.03999999910593033]

#### Predict on the new data

In [25]:
new = 'A'
a = char_to_int[new]
a = np.array(a)
a = np.reshape(a, (1,1,1))
a.shape

x_test = a / 25
x_test

pred = model.predict(x_test)
pred = pred.argmax(axis=1)
int_to_char[pred[0]]

1/1 [==============================] - 1s 596ms/step


'A'

In [26]:
def predict(char):
    a = char_to_int[char]
    a = np.array(a)
    a = np.reshape(a, (1,1,1))
    x_test = a / 25
    pred = model.predict(x_test)
    pred = pred.argmax(axis=1)
    return int_to_char[pred[0]]


In [27]:
predict('A')

1/1 [==============================] - 0s 41ms/step


'A'

In [28]:
predict('E')

1/1 [==============================] - 0s 28ms/step


'Z'

In [29]:
seq_length = 3

In [30]:
x = []
y = []
for i in range(len(alphabets)-seq_length):
    seq_in = alphabets[i:i+seq_length]
    seq_out = alphabets[i+seq_length]
    x.append([char_to_int[char] for char in seq_in])
    y.append([char_to_int[seq_out]])
    print(seq_in, '->', seq_out)

ABC -> D
BCD -> E
CDE -> F
DEF -> G
EFG -> H
FGH -> I
GHI -> J
HIJ -> K
IJK -> L
JKL -> M
KLM -> N
LMN -> O
MNO -> P
NOP -> Q
OPQ -> R
PQR -> S
QRS -> T
RST -> U
STU -> V
TUV -> W
UVW -> X
VWX -> Y
WXY -> Z


In [31]:
x

[[0, 1, 2],
 [1, 2, 3],
 [2, 3, 4],
 [3, 4, 5],
 [4, 5, 6],
 [5, 6, 7],
 [6, 7, 8],
 [7, 8, 9],
 [8, 9, 10],
 [9, 10, 11],
 [10, 11, 12],
 [11, 12, 13],
 [12, 13, 14],
 [13, 14, 15],
 [14, 15, 16],
 [15, 16, 17],
 [16, 17, 18],
 [17, 18, 19],
 [18, 19, 20],
 [19, 20, 21],
 [20, 21, 22],
 [21, 22, 23],
 [22, 23, 24]]

In [32]:
y_new = to_categorical(y)

In [33]:
len(x)

23

In [34]:
np.array(x).shape

(23, 3)

In [35]:
x = np.reshape(x, (23,seq_length,1))
x.shape

(23, 3, 1)

In [36]:
x = x / 25
x

array([[[0.  ],
        [0.04],
        [0.08]],

       [[0.04],
        [0.08],
        [0.12]],

       [[0.08],
        [0.12],
        [0.16]],

       [[0.12],
        [0.16],
        [0.2 ]],

       [[0.16],
        [0.2 ],
        [0.24]],

       [[0.2 ],
        [0.24],
        [0.28]],

       [[0.24],
        [0.28],
        [0.32]],

       [[0.28],
        [0.32],
        [0.36]],

       [[0.32],
        [0.36],
        [0.4 ]],

       [[0.36],
        [0.4 ],
        [0.44]],

       [[0.4 ],
        [0.44],
        [0.48]],

       [[0.44],
        [0.48],
        [0.52]],

       [[0.48],
        [0.52],
        [0.56]],

       [[0.52],
        [0.56],
        [0.6 ]],

       [[0.56],
        [0.6 ],
        [0.64]],

       [[0.6 ],
        [0.64],
        [0.68]],

       [[0.64],
        [0.68],
        [0.72]],

       [[0.68],
        [0.72],
        [0.76]],

       [[0.72],
        [0.76],
        [0.8 ]],

       [[0.76],
        [0.8 ],
        [0.84]],



In [37]:
len(x)

23

In [38]:
model = Sequential()

Dense(25, input_shape=(3,1), activation='relu'))
LSTM(128))
Dense(26, activation = 'softmax'))

In [39]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
model.fit(x,y_new, epochs=1000, batch_size=1)

Epoch 1/1000
23/23 [==============================] - 3s 7ms/step - loss: 3.2810 - accuracy: 0.0435
Epoch 2/1000
23/23 [==============================] - 0s 7ms/step - loss: 3.2468 - accuracy: 0.0435
Epoch 3/1000
23/23 [==============================] - 0s 8ms/step - loss: 3.2288 - accuracy: 0.0435
Epoch 4/1000
23/23 [==============================] - 0s 7ms/step - loss: 3.2033 - accuracy: 0.0435
Epoch 5/1000
23/23 [==============================] - 0s 8ms/step - loss: 3.1661 - accuracy: 0.0435
Epoch 6/1000
23/23 [==============================] - 0s 6ms/step - loss: 3.1211 - accuracy: 0.0000e+00
Epoch 7/1000
23/23 [==============================] - 0s 7ms/step - loss: 3.0717 - accuracy: 0.0000e+00
Epoch 8/1000
23/23 [==============================] - 0s 7ms/step - loss: 3.0225 - accuracy: 0.0000e+00
Epoch 9/1000
23/23 [==============================] - 0s 7ms/step - loss: 2.9676 - accuracy: 0.0435
Epoch 10/1000
23/23 [==============================] - 0s 7ms/step - loss: 2.9092 - accu

In [ ]:
model.evaluate(x, y_new, batch_size=1)

23/23 [==============================] - 1s 5ms/step - loss: 0.1267 - accuracy: 1.0000


[0.12671154737472534, 1.0]

In [ ]:
def predict(seq):
    a = []
    a.append([char_to_int[char] for char in seq])
    a = np.array(a)
    a = np.reshape(a, (1,3,1))
    x_test = a / 25
    pred = model.predict(x_test)
    pred = pred.argmax(axis=1)
    return int_to_char[pred[0]]


In [ ]:
predict('ABC')

1/1 [==============================] - 0s 29ms/step


'F'

In [ ]:
predict('WXY')

1/1 [==============================] - 0s 53ms/step


'Z'

In [ ]:
predict('LMN')